In [166]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

Чтобы ответить на вопрос, какие слова, фразы используются наиболее часто за последние 6 месяцев, выгрузим из базы данных StackOverflow Query набор комментариев, написанный за требуемый промежуток времени. Проблема заключается в том, что под заданное требование подходит более 2,5 млн записей. Поэтому возьмем не весь набор, а только часть. Чтобы это можно было обработать на домашнем компьютере, я взял всего лишь 1% от всех данных (26к комментариев).

Соответствующий запрос в базу данных:

select CreationDate, Text from Comments tablesample(1 percent)
where CreationDate >= DATEADD(month, -6, GETDATE())

Считаем данные и посмотрим немного на них.


In [167]:
data = pd.read_csv('QueryResults.csv')
data

,CreationDate,Text
0,2021-09-16 15:32:22,Welcome to Stack Overflow! In order to get the...
1,2021-09-16 15:32:23,Ok so how can I change the format? The dates a...
2,2021-09-16 15:32:28,"Try now mate, pls , i updated answer"
3,2021-09-16 15:32:38,I'm guessing that the HTMLness bit is set if t...
4,2021-09-16 15:32:42,"Thanks for the clarification, John"
...,...,...
26523,2022-03-13 03:23:46,"In your repo, this `func compositionLayerInstr..."
26524,2022-03-13 03:23:50,@QueenoftheNorth The two parameters are indica...
26525,2022-03-13 03:24:13,"The code is buggy. I tried `build_path(""http:/..."
26526,2022-03-13 03:24:18,What about start thinking with the following t...


In [168]:
data.dtypes

CreationDate    object
Text            object
dtype: object

In [169]:
data.columns

Index(['CreationDate', 'Text'], dtype='object')

In [170]:
print(data.shape)
for feature in data.columns:
    print(feature.ljust(27), data[feature].nunique())

(26528, 2)
CreationDate                24057
Text                        26078


Похоже есть повторяющиеся комментарии :)

In [171]:
data[data.isnull().any(axis=1)]

,CreationDate,Text


In [172]:
X = data.to_numpy()[:, 1]

Наивных подход. Пройдемся по всем сообщениям и посмотрим число вхождений всех уникальных слов. Для этого уберем все стоп-слова, переведем все в нижний регистр и уберем все знаки препинания.

In [173]:
def trend_words(X, d_size = 100):
    d = {}
    stop_words = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')    # оставляет только слова, без знаков припинания
    #stemmer = WordNetLemmatizer()    # приводит слова к нормальной форме
    for line in X:
        s = [w for w in tokenizer.tokenize(line.lower()) if w not in stop_words]
        for word in s:
            if word in d:
                d[word] += 1
            else:
                d[word] = 1
    word_freq_list = sorted(d.items(), key=lambda x: x[1], reverse=True)
    return pd.DataFrame(word_freq_list[:d_size]).rename(columns={0: 'words', 1: 'freq'})


Посмотрим на самые частые слова

In [174]:
trend_words(X)[:30]

,words,freq
0,https,3820
1,code,3438
2,use,3116
3,com,3004
4,answer,2753
5,question,2559
6,using,2230
7,need,2165
8,like,2106
9,thanks,2095


Видно, что это просто самые частые слова, которые используются в речи. Заметим, что очень много дается ссылок ('https' и 'stackoverflow')
Можно попробовать собирать статистику не просто для всех слов, а каким-то образом их отбирать, чтобы они были важные и считать число вхождений уже для них. Для этого воспользуемся библиотекой rake_nltk. Кроме поиска важных слов, с помощью этой библиотеки можно искать ключевые фразы. Тоже посчитаем их статистику. Чтобы все работало лучше, необходимо немного чистить сообщения. Для этого воспользуемся RegexpTokenizer.

In [175]:
from rake_nltk import Rake

In [176]:
def add_to_dict(someting, d):
    if someting:
        if isinstance(someting, list):
            for word in someting:
                if word in d:
                    d[word] += 1
                else:
                    d[word] = 1
        else:
            if someting in d:
                d[someting] += 1
            else:
                d[someting] = 1


def super_trend_words(X, size = 100):
    d_phrases = {}
    d_words = {}
    stop_words = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')    # оставляет только слова, без знаков припинания
    # stemmer = WordNetLemmatizer()    # приводит слова к нормальной форме

    for line in X:
        s = [w for w in tokenizer.tokenize(line.lower()) if w not in stop_words]
        new_line = ' '.join(map(str, s))
        rake = Rake()
        rake.extract_keywords_from_text(new_line)
        key_phrase = rake.get_ranked_phrases()
        if key_phrase:
            key_phrase = key_phrase[:3]
        else:
            key_phrase = ''
        key_words = list(rake.get_word_degrees().keys())[:3]
        add_to_dict(key_words, d_words)
        add_to_dict(key_phrase, d_phrases)
    word_freq_list = sorted(d_words.items(), key=lambda x: x[1], reverse=True)
    phrase_freq_list = sorted(d_phrases.items(), key=lambda x: x[1], reverse=True)
    return np.array(word_freq_list[:size]), np.array(phrase_freq_list[:size])


In [177]:
words, phrases = super_trend_words(X)


Посмотрим на важные слова. По ссути, это опять же слова, использующиеся в обычной речи.

In [178]:
pd.DataFrame(words[:30]).rename(columns={0: 'words', 1: 'freq'})

,words,freq
0,thanks,1436
1,answer,1093
2,thank,1070
3,please,947
4,question,829
5,use,822
6,yes,804
7,code,750
8,https,623
9,need,576


Посмотрим на фразы. Видно, что из того, что удалось обнаружить, мы получили какие-то подсказки для пользователей, оставляющих комментарии :)
Еще есть небольшое количество фраз, которые тоже используются в повседневной речи

In [180]:
pd.DataFrame(phrases[:30]).rename(columns={0: 'words', 1: 'freq'})

,words,freq
0,please provide enough code others better under...,90
1,please clarify specific problem provide additi...,80
2,answer could improved additional supporting in...,76
3,currently written answer unclear please edit a...,52
4,please edit question limit specific problem en...,40
5,thank much,18
6,please trim code make easier find problem foll...,10
7,question,10
8,work,9
9,welcome,7


In [181]:
pd.DataFrame(phrases[:30]).rename(columns={0: 'words', 1: 'freq'})

,words,freq
0,please provide enough code others better under...,90
1,please clarify specific problem provide additi...,80
2,answer could improved additional supporting in...,76
3,currently written answer unclear please edit a...,52
4,please edit question limit specific problem en...,40
5,thank much,18
6,please trim code make easier find problem foll...,10
7,question,10
8,work,9
9,welcome,7


In [182]:
phrases[:30]

array([['please provide enough code others better understand reproduce problem',
        '90'],
       ['please clarify specific problem provide additional details highlight exactly need currently written hard tell exactly asking',
        '80'],
       ['answer could improved additional supporting information please edit add details citations documentation others confirm answer correct find information write good answers help center help answer',
        '76'],
       ['currently written answer unclear please edit add additional details help others understand addresses question asked find information write good answers help center help answer',
        '52'],
       ['please edit question limit specific problem enough detail identify adequate answer',
        '40'],
       ['thank much', '18'],
       ['please trim code make easier find problem follow guidelines create minimal reproducible example https stackoverflow com help minimal reproducible example',
        '10'],
       ['ques

В результате, я смог найти множество слов, которые люди говорят в повседневном обсуждении вопросов на stack overflow